In [141]:
import requests
from requests_futures.sessions import FuturesSession
from requests_futures import sessions
import json
import psycopg2
from psycopg2 import Error
import pandas.io.sql as psql
import pandas as pd
from datetime import datetime, timedelta
import time

In [142]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


def check(params):
    session = sessions.FuturesSession(max_workers=6)
    futures = [session.get(url="https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/findByPin",
                           headers=headers, params=p) for p in params]
    for f in futures:
        f.result()

    session.close()

    return futures


In [143]:

while True:
    time.sleep(300)
    try:
        conn = psycopg2.connect(dbname='dfpglck7sd4deq', user='vfpfcrcywbbppp',
                                password='2306588cdf3b2662603049a475863835f2ac89cdd52af890323a181d0405601c',
                                host='ec2-52-0-114-209.compute-1.amazonaws.com', port='5432', sslmode='require')
        cur = conn.cursor()

        print('Connected!')

    except (Exception, Error) as error:
        print("Error while connecting to PostgreSQL", error)

    table = psql.read_sql('SELECT * FROM home_customer', conn)

    pins = table.pin.unique()
    avail = [0 for i in range(len(pins))]

    current = []
    for pin in pins:
        current.append(table[table['pin'] == pin]['flag2'].values[0])

    params = []
    date = datetime.today()

    for pin in pins:
        params.append(
            {'pincode': str(pin), 'date': str(date.strftime("%d-%m-%Y"))})

    responses = check(params)

    for i in range(len(pins)):
        res = responses[i].result().json()

        if list(res.keys())[0] == 'sessions':
            if len(res['sessions']) != 0:
                for session in res['sessions']:
                    for j in range(len(session)):
                        if session['available_capacity'] > 0 and session['min_age_limit'] == 18:
                            avail[i] = 1

        elif list(res.keys())[0] == 'centers':
            if len(res['centers']) != 0:
                for center in res['centers']:
                    for j in range(len(center['sessions'])):
                        if center['sessions'][j]['available_capacity'] > 0 and center['sessions'][j]['min_age_limit'] == 18:
                            avail[i] = 1


    for i in range(len(pins)):
        if avail[i] != current[i]:
            if avail[i] > current[i]:
                emails = table[table['pin'] == pins[i]]['email'].values

                for email in emails:
                    print(f'{pins[i]} {email} Sent')
                    requests.post(
                        "https://api.mailgun.net/v3/whenismyvaccine.in/messages",
                        auth=("api", "83c8481eed353ca9d76bbdd3101a2b33-2a9a428a-5bb25d17"),
                        data={"from": "When is my vaccine? <alerts@whenismyvaccine.in>",
                            "to": email,
                            "subject": "Thank for registering to WhenIsMyVaccine",
                            "text": f"Hello {email}, \n Vaccines are available in your area. For pin: {pins[i]}"})

                cur.execute(
                    f'UPDATE home_customer SET flag1=1, flag2=1 WHERE pin={pins[i]};')
                conn.commit()

            elif avail[i] < current[i]:
                print(f'{pins[i]} {email} Unflagged')
                cur.execute(
                    f'UPDATE home_customer SET flag1=0, flag2=0 WHERE pin={pins[i]};')
                conn.commit()

            elif avail[i] == 1:
                print(f'{pins[i]} {email}')

                emails = table[table['pin'] == pins[i] and (
                    table['flag2'] == 1 and table['flag1'] == 0)]['email'].values

                for email in emails:
                    print(f'{pins[i]} {email}')
                    requests.post(
                        "https://api.mailgun.net/v3/whenismyvaccine.in/messages",
                        auth=(
                            "api", "83c8481eed353ca9d76bbdd3101a2b33-2a9a428a-5bb25d17"),
                        data={"from": "When is my vaccine? <alerts@whenismyvaccine.in>",
                            "to": email,
                            "subject": "Thank for registering to WhenIsMyVaccine",
                            "text": f"Hello {email}, \n Vaccines are available in your area. For pin: {pins[i]}"})

                cur.execute(
                    f'UPDATE home_customer SET flag1=1 WHERE flag1=0 and pin={pins[i]};')
                conn.commit()


    


Connected!
Connected!
121003 shivamshg19@gmail.com
Connected!


KeyboardInterrupt: 

In [70]:
try:
    conn = psycopg2.connect(dbname='dfpglck7sd4deq', user='vfpfcrcywbbppp',
                            password='2306588cdf3b2662603049a475863835f2ac89cdd52af890323a181d0405601c',
                            host='ec2-52-0-114-209.compute-1.amazonaws.com', port='5432', sslmode='require')
    cur = conn.cursor()

    print('Connected!')

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

table = psql.read_sql('SELECT * FROM home_customer', conn)

Connected!


In [71]:
table

,id,pin,flag1,flag2,email,phone
0,10156270-8400-418d-a2b4-e4e24b39d10a,400064,0,0,strayking2811@gmail.com,8169011347
1,6e32aa33-64b3-4d5f-81a9-bc8d2b424d1a,324009,0,0,anshul.singhvi84@nmims.edu.in,8112248828
2,e4aeef96-b2f9-4d28-adcf-882eb6ad0bb9,226006,0,0,saahilyadav7@gmail.com,7985820523
3,914b12c5-4790-465f-b481-0b6020daea37,400706,0,0,archit10.1999@gmail.com,+919702070
4,d8585f16-b263-41bc-9f20-1986041f955e,452001,0,0,chikkiajmera@gmail.com,9511290293
...,...,...,...,...,...,...
65,8f7a6f3c-81dc-4fc7-8c55-ea767aa052a3,451447,0,0,radhika87308@gmail.com,9522187308
66,fe79d4b5-8f6e-4132-84e9-1a93a5085982,451449,0,0,radhika87308@gmail.com,9522187308
67,c00bf977-b067-442f-8d3e-5abea78971f0,451770,0,0,radhika87308@gmail.com,9522187308
68,3a114c6c-2ad9-41fb-9a4b-07662257681d,444001,0,0,atuld801@gmail.com,9960138802


In [73]:
pins = table.pin.unique()
avail = [0 for i in range(len(pins))]

current = []

for pin in pins:
    current.append(table[table['pin'] == pin]['flag2'].values[0])

In [82]:
params = []
date = datetime.today()

for pin in pins:
    params.append(
        {'pincode': str(pin), 'date': str(date.strftime("%d-%m-%Y"))})

In [90]:
responses = check(params)

for i in range(len(pins)):
    res = responses[i].result().json()

    if list(res.keys())[0] == 'sessions':
        if len(res['sessions']) != 0:
            for session in res['sessions']:
                for j in range(len(session)):
                    if session['available_capacity'] > 0 and session['min_age_limit'] == 18:
                        avail[i] = 1
    
    elif list(res.keys())[0] == 'centers':
                if len(res['centers']) != 0:
                    for center in res['centers']:
                        for j in range(len(center['sessions'])):
                            if center['sessions'][j]['available_capacity'] > 0 and center['sessions'][j]['min_age_limit'] == 18:
                                avail[i] = 1

In [127]:
for i in range(len(pins)):
    print(i)
    
    if avail[i] != current[i]:
        if avail[i] > current[i]:
            emails = table[table['pin'] == pins[i]]['email'].values

            for email in emails:
                print(f'{pins[i]} {email}')
                '''requests.post(
                    "https://api.mailgun.net/v3/whenismyvaccine.in/messages",
                    auth=("api", "83c8481eed353ca9d76bbdd3101a2b33-2a9a428a-5bb25d17"),
                    data={"from": "When is my vaccine? <alerts@whenismyvaccine.in>",
                        "to": email,
                        "subject": "Thank for registering to WhenIsMyVaccine",
                        "text": f"Hello {email}, \n Vaccines are available in your area. For pin: {pins[i]}"})'''

            cur.execute(
                f'UPDATE home_customer SET flag1=1, flag2=1 WHERE pin={pins[i]};')
            conn.commit()

        elif avail[i] < current[i]:
            cur.execute(
                f'UPDATE home_customer SET flag1=0, flag2=0 WHERE pin={pins[i]};')
            conn.commit()

    elif avail[i] == 1:
        print(f'{pins[i]} {email}')
        
        emails = table[table['pin'] == pins[i] and (
            table['flag2'] == 1 and table['flag1'] == 0)]['email'].values

        for email in emails:
            requests.post(
                "https://api.mailgun.net/v3/whenismyvaccine.in/messages",
                auth=(
                    "api", "83c8481eed353ca9d76bbdd3101a2b33-2a9a428a-5bb25d17"),
                data={"from": "When is my vaccine? <alerts@whenismyvaccine.in>",
                    "to": email,
                    "subject": "Thank for registering to WhenIsMyVaccine",
                    "text": f"Hello {email}, \n Vaccines are available in your area. For pin: {pins[i]}"})

        cur.execute(
            f'UPDATE home_customer SET flag1=1 WHERE flag1=0 and pin={pins[i]};')
        conn.commit()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
122001 7ganushkam1999@gmail.com
44
45
46
47
48
49
50
51
52


In [122]:
table[table['pin'] == pins[43]]['email']

55    7ganushkam1999@gmail.com
Name: email, dtype: object

In [115]:
pins[43]

122001